In [5]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
data = pd.read_csv("dataset_2024_2025_updated.csv")
data.head()

,comment_id,self_text,post_id,post_title,author_name,user_is_verified,post_created_time,clean_text,side
0,mevinyg,"This is all one side, no?",1iyjjde,Trump: ‘I know some Russian oligarchs that are...,MycologistRecent8959,True,2025-02-26 09:15:47,"This is all one side, no?",Rusia
1,mevilft,Renaming MAGA to MAR (Make America Russia)?,1iym4pu,Trump says Ukraine’s Zelenskyy to sign ‘very b...,PointedSticks,True,2025-02-26 12:13:33,Renaming MAGA to MAR (Make America Russia)?,Rusia
2,mevildy,https://www.washingtonpost.com/politics/2025/0...,1iyl54v,Trump says Russian oligarchs could be eligible...,traumfisch,True,2025-02-26 11:11:23,http,Rusia
3,mevikqu,The US would be alone in those attacks anyway....,1iyjwrk,"""Putin waits for Trump to negotiate Ukraine wi...",Whatcanyado420,True,2025-02-26 09:44:03,The US would be alone in those attacks anyway....,Rusia
4,mevijjm,Withholding elections during a total war for y...,1iyjsu5,UA POV: Oleksiy Kosach - Trump – not Zelensky...,kirotheavenger,True,2025-02-26 09:35:47,Withholding elections during a total war for y...,Rusia


In [7]:
data['post_created_time'] = pd.to_datetime(data['post_created_time'])

In [8]:
data.isna().sum()

comment_id           0
self_text            0
post_id              0
post_title           0
author_name          0
user_is_verified     0
post_created_time    0
clean_text           0
side                 0
dtype: int64

In [9]:
import tensorflow as tf
from transformers import pipeline

# sentiment analysis task with twitter roberta model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

Device set to use cuda:0


In [14]:
import pandas as pd
from datasets import Dataset
from transformers import pipeline, AutoTokenizer

# Pastikan 'clean_text' hanya berisi string dan tidak ada nilai NaN
data['clean_text'] = data['clean_text'].fillna("")  # Ganti NaN dengan string kosong

# Muat tokenizer untuk model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Potong teks ke panjang maksimum 512 token
def truncate_text(text, max_length=512):
    return tokenizer.decode(tokenizer.encode(text, truncation=True, max_length=max_length))

data['clean_text'] = data['clean_text'].apply(truncate_text)

# Konversi DataFrame ke Dataset
dataset = Dataset.from_pandas(data)

# Inisialisasi pipeline dengan truncation dan max_length
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer=tokenizer,  # Gunakan tokenizer yang sama
    truncation=True,
    max_length=512
)

# Fungsi untuk memproses batch
def process_batch(batch):
    try:
        results = sentiment_pipeline(batch['clean_text'])
        return {
            "sentiment": [result['label'] for result in results],
            "sentiment_score": [result['score'] for result in results]
        }
    except Exception as e:
        print(f"Error processing batch: {batch['clean_text']}")
        raise

# Terapkan fungsi ke dataset dengan batch processing
batch_size = 64  # Sesuaikan ukuran batch sesuai kapasitas GPU
dataset = dataset.map(process_batch, batched=True, batch_size=batch_size)

# Konversi kembali ke DataFrame
data = dataset.to_pandas()

# Map model labels ke kategori yang lebih bermakna
label_mapping = {
    'LABEL_0': 'Negative',
    'LABEL_1': 'Neutral',
    'LABEL_2': 'Positive'
}
data['sentiment'] = data['sentiment'].map(label_mapping)

# Simpan hasil ke file CSV
data.to_csv('dataset_with_sentiment.csv', index=False)

# Tampilkan hasil
print("Dataset berhasil diperbarui dan disimpan sebagai dataset_with_sentiment.csv")
print(data.head())

Device set to use cuda:0


Map:   0%|          | 0/1169005 [00:00<?, ? examples/s]

Dataset berhasil diperbarui dan disimpan sebagai dataset_with_sentiment.csv
  comment_id                                          self_text  post_id  \
0    mevinyg                          This is all one side, no?  1iyjjde   
1    mevilft        Renaming MAGA to MAR (Make America Russia)?  1iym4pu   
2    mevildy  https://www.washingtonpost.com/politics/2025/0...  1iyl54v   
3    mevikqu  The US would be alone in those attacks anyway....  1iyjwrk   
4    mevijjm  Withholding elections during a total war for y...  1iyjsu5   

                                          post_title           author_name  \
0  Trump: ‘I know some Russian oligarchs that are...  MycologistRecent8959   
1  Trump says Ukraine’s Zelenskyy to sign ‘very b...         PointedSticks   
2  Trump says Russian oligarchs could be eligible...            traumfisch   
3  "Putin waits for Trump to negotiate Ukraine wi...        Whatcanyado420   
4  UA POV:  Oleksiy Kosach - Trump – not Zelensky...        kirotheavenger   

In [22]:
dataset = pd.read_csv("dataset_with_sentiment.csv")

In [28]:
dataset.loc[(dataset['side'] == 'Rusia') & (dataset['sentiment'] == 'Negative')].head()

,comment_id,self_text,post_id,post_title,author_name,user_is_verified,post_created_time,clean_text,side,sentiment,sentiment_score
3,mevikqu,The US would be alone in those attacks anyway....,1iyjwrk,"""Putin waits for Trump to negotiate Ukraine wi...",Whatcanyado420,True,2025-02-26 09:44:03,<s>The US would be alone in those attacks anyw...,Rusia,Negative,0.66
4,mevijjm,Withholding elections during a total war for y...,1iyjsu5,UA POV: Oleksiy Kosach - Trump – not Zelensky...,kirotheavenger,True,2025-02-26 09:35:47,<s>Withholding elections during a total war fo...,Rusia,Negative,0.73
5,meviixk,"They don't take to streets like europeans, I m...",1iyl54v,Trump says Russian oligarchs could be eligible...,CneusPompeius,True,2025-02-26 11:11:23,"<s>They don't take to streets like europeans, ...",Rusia,Negative,0.79
6,mevihno,I don't think any alleged/assumed security gua...,1iyhakh,"US, Ukraine agree to terms on mineral deal, Uk...",East_Type_1136,True,2025-02-26 06:32:52,<s>I don't think any alleged/assumed security ...,Rusia,Negative,0.60
8,mevih55,"Horrible, reprehensible, and pretty much par f...",1iyilcw,What are your thoughts on Trump calling Zelens...,Flyers45432,True,2025-02-26 08:05:06,"<s>Horrible, reprehensible, and pretty much pa...",Rusia,Negative,0.95


In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169005 entries, 0 to 1169004
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   comment_id         1169005 non-null  object 
 1   self_text          1169005 non-null  object 
 2   post_id            1169005 non-null  object 
 3   post_title         1169005 non-null  object 
 4   author_name        1169005 non-null  object 
 5   user_is_verified   1169005 non-null  bool   
 6   post_created_time  1169005 non-null  object 
 7   clean_text         1169005 non-null  object 
 8   side               1169005 non-null  object 
 9   sentiment          1169005 non-null  object 
 10  sentiment_score    1169005 non-null  float64
dtypes: bool(1), float64(1), object(9)
memory usage: 90.3+ MB


In [30]:
dataset['post_created_time'] = pd.to_datetime(dataset['post_created_time'])

In [32]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169005 entries, 0 to 1169004
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   comment_id         1169005 non-null  object        
 1   self_text          1169005 non-null  object        
 2   post_id            1169005 non-null  object        
 3   post_title         1169005 non-null  object        
 4   author_name        1169005 non-null  object        
 5   user_is_verified   1169005 non-null  bool          
 6   post_created_time  1169005 non-null  datetime64[ns]
 7   clean_text         1169005 non-null  object        
 8   side               1169005 non-null  object        
 9   sentiment          1169005 non-null  object        
 10  sentiment_score    1169005 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(1), object(8)
memory usage: 90.3+ MB
